## MOVIES RECOMENDATION SYSTEM PROJECT
#### GROUP 3
##### Group Members:
      Gideon Ochieng
      Lorna Wangui
      Ann Mwangi
      Charles Odhiambo
      Victor Masinde


### Project Overview
An international powerhouse, the film business brings in billions of dollars every year. According to Statista, the worldwide box office brought in an incredible $42.5 billion in 2019 alone. Film production, which include studios, independent producers, and streaming services, is the creative centre where motion pictures, television series, and other visual content are made. These works are marketed and delivered to theatres, streaming platforms, and other locations by the distribution division. The term "exhibition" describes the showing of films and television series in theatres. The allure of films is universal, bringing people from different origins together. Despite this widespread appeal, our individual film preferences are distinctive, ranging from romance to sci-fi to thrillers, and frequently focussing on our favourite actors and filmmakers.

### Business Understanding
#### Real-World Problem
In an era where streaming platforms dominate the entertainment landscape, users often face difficulty in selecting movies due to the overwhelming number of choices available. This leads to "choice paralysis," where the sheer volume of options prevents users from making satisfying decisions. A recommendation system can address this by providing personalized movie suggestions based on a user’s past ratings, making their movie-watching experience more enjoyable and efficient.

#### Stakeholders
Primary Stakeholders:

End Users: Individuals who need personalized recommendations to find movies they are likely to enjoy.
Streaming Platforms: Businesses looking to enhance user engagement and retention by offering tailored recommendations.
Secondary Stakeholders:

Movie Studios: Studios seeking insights into audience preferences to guide future productions.
Market Researchers: Analysts studying consumer behavior through movie preferences and trends.
Project Value
The project will solve the problem of overwhelming choice by providing users with a personalized top-5 list of movie recommendations. It will:

Enhance user satisfaction by reducing time spent searching for movies.
Improve user engagement for streaming platforms, leading to higher retention rates.
Help streaming platforms and studios understand user preferences, informing content strategies and acquisitions.
Business Goals
Create a recommendation system using collaborative filtering to predict user preferences.
Provide an easy-to-use interface where users can input ratings for movies they've watched.
Ensure the system addresses the "cold start" problem for new users by incorporating content-based techniques.

Conclusion

The recommendation system will make movie selection faster and more satisfying for users, while also providing business value to streaming platforms and studios by increasing engagement and understanding consumer preferences.